In [4]:
import os
import string

import numpy as np
import pandas as pd

import re
import nltk
import pyLDAvis as pyLDAvis
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob
from langdetect import detect
from nltk import word_tokenize

from autocorrect import Speller


from nltk.stem import PorterStemmer

spell = Speller(lang='en')

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
stopwords.words('english')
nltk.download('averaged_perceptron_tagger')




[nltk_data] Downloading package punkt to /Users/fahri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/fahri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/fahri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /Users/fahri/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fahri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
#reading files
path = "/Users/fahri/Desktop/review_polarity/txt_sentoken/neg"

list_ = []
# Change the directory
os.chdir(path)
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
        list_.append(data)

# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"

        # call read text file function
        read_text_file(file_path)


df1= pd.DataFrame({'Content':list_})
path = "/Users/fahri/Desktop/review_polarity/txt_sentoken/pos"
list_ = []

os.chdir(path)

# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"

        # call read text file function
        read_text_file(file_path)


df2= pd.DataFrame({'Content':list_})

In [6]:
#concatenation of neg and pos
frames = [df1,df2]
df = pd.concat(frames, ignore_index=True)


#for train test copy this column
dfTrain = pd.DataFrame()
dfTrain = df.copy()


In [7]:
sent_list = []
for i in range(2000):
    sent_list.append(df['Content'][i])
df_sent = pd.DataFrame({'Content': sent_list})

df.head()

,Content
0,bad . bad . \nbad . \nthat one word seems to p...
1,isn't it the ultimate sign of a movie's cinema...
2,""" gordy "" is not a movie , it is a 90-minute-..."
3,disconnect the phone line . \ndon't accept the...
4,when robert forster found himself famous again...


In [8]:
df['Content'] = df['Content'].str.replace('\n', ' ')  #remove new line characters
df.head()

,Content
0,bad . bad . bad . that one word seems to pre...
1,isn't it the ultimate sign of a movie's cinema...
2,""" gordy "" is not a movie , it is a 90-minute-..."
3,disconnect the phone line . don't accept the ...
4,when robert forster found himself famous again...


In [9]:

import spacy
nlp = spacy.load("en_core_web_sm")
from spacy.tokens import Doc
def stringTolist(s):

    # initialize an empty string
    str1 = ""
    # return string
    return (str1.join(s))

def pre_process_text(doc,j):
    # Generate a new list of tokens here
    new_words = doc[j].lemma_
    new_doc = Doc(doc.vocab, words=new_words)
    return new_doc

for i in range(2000):
    sentence = df['Content'][i]
    doc = nlp(sentence)
    word_list = []

    for j in range(len(doc)):


        if str(doc[j]) != str(doc[j].lemma_):
            a = stringTolist(str(pre_process_text(doc,j)).replace(" ",""))


            #print(counter)
           # print(f"{str(token):>20} : {str(token.lemma_)}")
        else :
            a = stringTolist(str(doc[j]))

        word_list.append(a)

    df['Content'][i] = word_list

df['Content'].head()

ValueError: [E088] Text of length 1037950616 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [ ]:
#df['Content'] = df['Content'].apply(lambda x: ''.join(TextBlob(x).correct()))
# too long to run

In [ ]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

df['Content'] = df['Content'].apply(lambda x: reduce_lengthening(str(x)))

In [ ]:
stop_words = stopwords.words('English')
wnl = WordNetLemmatizer()


for i in range(2000):
    df['Content'][i] = re.sub(r'[^a-zA-Z]', ' ', str(df['Content'][i])) # punctuation and numbers
    df['Content'][i] = re.sub(r'^\s*|\s\s*', ' ', df['Content'][i]).strip().lower() #double whitespace

    df['Content'][i] = re.split("\W+", df['Content'][i] ) #tokenize
    df['Content'][i] = [word for word in df['Content'][i] if word not in stop_words] # stop words


In [ ]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

for i in range(2000):
    sent = df['Content'][i]
    sent = " ".join(w for w in nltk.wordpunct_tokenize(str(sent)) if w.lower() in words or not w.isalpha())

df.head()

In [ ]:
df['number_of_words'] = df['Content'].apply(lambda x : len(TextBlob(str(x)).words))

df['polarity'] = df['Content'].apply(lambda x : TextBlob(str(x)).sentiment.polarity)

df['subjectivity'] = df['Content'].apply(lambda x : TextBlob(str(x)).sentiment.subjectivity)

df['language'] = df['Content'].apply(lambda x : detect(str(x)))

df.head()

In [ ]:
def listToString(s):

    # initialize an empty string
    str1 = " "

    # return string
    return (str1.join(s))

corpus = []
for i in range(2000):
    a = listToString(df['Content'][i])
    corpus.append(a)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


bag_of_words_model = CountVectorizer()
bag_of_word_df = pd.DataFrame(bag_of_words_model.fit_transform(corpus).todense())
bag_of_word_df.columns = sorted(bag_of_words_model.vocabulary_)

bag_of_word_df.head()

In [ ]:
bag_of_words_model_small = CountVectorizer(max_features=10)
bag_of_word_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())
bag_of_word_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

bag_of_word_df_small.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_model = TfidfVectorizer()
tfidf_df = pd.DataFrame(tfidf_model.fit_transform(corpus).todense())
tfidf_df.columns = sorted(tfidf_model.vocabulary_)

tfidf_df.head()

In [ ]:
tfidf_model_small = TfidfVectorizer(max_features=10)
tfidf_df_small = pd.DataFrame(tfidf_model_small.fit_transform(corpus).todense())
tfidf_df_small.columns = sorted(tfidf_model_small.vocabulary_)

tfidf_df_small.head()

In [ ]:
tfidf_df_small.head

In [ ]:
bag_of_words_model_small2 = CountVectorizer(max_features=50)
bag_of_word_df_small2 = pd.DataFrame(bag_of_words_model_small2.fit_transform(corpus).todense())
bag_of_word_df_small2.columns = sorted(bag_of_words_model_small2.vocabulary_)

bag_of_word_df_small2.head()

In [ ]:
words50 = {}

for i in range(50):
    words50[((bag_of_word_df_small2.columns)[i])] = sum(bag_of_word_df_small2[((bag_of_word_df_small2.columns)[i])])

from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wc = WordCloud(background_color="blue",width=2000,height=1000, max_words=50,relative_scaling=0.5).generate_from_frequencies(words50)
plt.figure(figsize=(10,8))
plt.imshow(wc)

In [ ]:
from sklearn.cluster import KMeans

# initialize kmeans with 5 centroids
kmeans = KMeans(n_clusters=5, random_state=42)
# fit the model
kmeans.fit(bag_of_word_df)
# store cluster labels in a variable
clusters = kmeans.labels_

In [ ]:

from sklearn.decomposition import PCA

# initialize PCA with 2 components
pca = PCA(n_components=2, random_state=42)
# pass our X to the pca and store the reduced vectors into pca_vecs
pca_vecs = pca.fit_transform(bag_of_word_df)
# save our two dimensions into x0 and x1
x0 = pca_vecs[:, 0]
x1 = pca_vecs[:, 1]

In [ ]:
# assign clusters and pca vectors to our dataframe
df['cluster'] = clusters

df['x0'] = x0
df['x1'] = x1

In [ ]:
dfTrain['Content'] =  df['Content'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
from sklearn.model_selection import train_test_split
Xk = dfTrain['Content']
yk = df['cluster']
X_train, X_test, y_train, y_test = train_test_split(Xk,yk,test_size = 0.2,random_state = 6)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
clfk = Pipeline([('vect', tfidf_model), ('clf', KMeans(n_clusters=5))])

clfk.fit(X_train, y_train)

In [ ]:
# Check accuracy, precision,recall
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(classification_report(y_test, clfk.predict(X_test)))

In [ ]:
import seaborn as sns
# set image size
plt.figure(figsize=(12, 7))
# set a title
plt.title("TF-IDF + KMeans ", fontdict={"fontsize": 15})
# set axes names
plt.xlabel("X0", fontdict={"fontsize": 15})
plt.ylabel("X1", fontdict={"fontsize": 15})
# create scatter plot with seaborn, where hue is the class used to group the data
sns.scatterplot(data=df, x='x0', y='x1', hue='cluster', palette="Set2")
plt.show()

In [ ]:
df.head()

In [ ]:
sentiment = []
is_emotional = []

for i in range(2000):
    if df['polarity'][i] < 0:
        sentiment.append(0)
    else:
        sentiment.append(1)
    if df['subjectivity'][i] < 0.5:
        is_emotional.append(0)
    else:
        is_emotional.append(1)

In [ ]:
df['sentiment'] = sentiment
df['is_emotional'] = is_emotional


In [ ]:
from sklearn.model_selection import train_test_split
X = dfTrain['Content']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 6)


In [ ]:
# Connect LogisticRegression and TfidfVectorizer by Pineline

from sklearn.linear_model import LogisticRegression

clf = Pipeline([('vect', tfidf_model), ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

In [ ]:
# Check accuracy, precision,recall

print(classification_report(y_test, clf.predict(X_test)))

In [ ]:

# Sentiment_model evaluation by ROC Curves
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import metrics

fig = plt.figure(1, figsize = (5, 5))
y_pred_proba = clf.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc='lower right')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curves')
plt.show()

In [ ]:
from sklearn.svm import LinearSVC

#Linear SVC
classifier_svc = Pipeline([('tfidf',TfidfVectorizer()),
('clf',LinearSVC())])

classifier_svc.fit(X_train, y_train)

print(classification_report(y_test, classifier_svc.predict(X_test)))


In [ ]:
from sklearn.model_selection import train_test_split
X = dfTrain['Content']
y = df['is_emotional']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Random Forest
classifier_rf = Pipeline([('tfidf',TfidfVectorizer()), ('clf',RandomForestClassifier(bootstrap= False, criterion= 'entropy', n_estimators= 100))])
classifier_rf.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test, classifier_rf.predict(X_test)))

In [ ]:
fig = plt.figure(1, figsize = (5, 5))
y_pred_proba = classifier_rf.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc='lower right')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curves')
plt.show()

In [ ]:
#LDA
import gensim
from gensim import corpora


In [ ]:


tokenized = df['Content']
dictionary = corpora.Dictionary(tokenized)
dictionary.filter_extremes(no_below=1, no_above=0.8)
corpus2 = [dictionary.doc2bow(tokens) for tokens in tokenized]
print(corpus2[:1])

In [ ]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus2[:1]]

In [ ]:
#LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus2, num_topics = 5, id2word=dictionary, passes=15)
ldamodel.save('mOdel.gensim')
topics = ldamodel.print_topics(num_words=30)


In [ ]:
get_document_topics = ldamodel.get_document_topics(corpus2[6])
print(get_document_topics)

In [ ]:
#visualizing topics
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
lda_viz = gensim.models.ldamodel.LdaModel.load('mOdel.gensim')#load lda model

lda_display = gensimvis.prepare(lda_viz, corpus2, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

In [ ]:
df_pos = pd.DataFrame()
df_neg = pd.DataFrame()

df_neg['Content'] = df_sent['Content'].iloc[0:1000]
df_pos['Content'] = df_sent['Content'].iloc[1001:2000]



In [ ]:
neg_string = ""

for i in range(10):
    neg_string += df_neg['Content'][i]

neg_string

In [ ]:
pos_string = ""

for i in range(1001,1010):
    pos_string += df_pos['Content'][i]


In [ ]:
pos_string

In [ ]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def summarize(text, n):
    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    # Tokenize and vectorize each sentence
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(sentences).toarray()
    # Compute the similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = cosine_similarity(vectors[i].reshape(1, -1), vectors[j].reshape(1, -1))[0,0]
    # Run TextRank
    sentence_ranks = textrank(similarity_matrix)
    # Sort the sentence ranks
    ranked_sentences = sorted(((score, index) for index, score in enumerate(sentence_ranks)), reverse=True)
    # Extract the top-ranked sentences
    top_n_ranked_sentences = [ranked_sentences[i] for i in range(n)]
    top_n_ranked_sentences.sort(key=lambda x: x[1])
    # Extract the summary
    summary = " ".join([sentences[index] for score, index in top_n_ranked_sentences])
    return summary

def textrank(matrix):
    # Add a small constant to the matrix to avoid division by zero
    matrix += 1e-9
    # Compute the transition matrix
    transition_matrix = matrix / matrix.sum(axis=1)[:, None]
    # Initialize the sentence ranks with uniform probability
    sentence_ranks = np.ones(matrix.shape[0]) / matrix.shape[0]
    # Set the damping factor
    damping_factor = 0.85
    # Set the tolerance
    tolerance = 1e-4
    # Run the PageRank algorithm until convergence
    while True:
        next_sentence_ranks = (1 - damping_factor) + damping_factor * transition_matrix.T.dot(sentence_ranks)
        diff = np.abs(sentence_ranks - next_sentence_ranks)
        if diff.max() < tolerance:
            break
        sentence_ranks = next_sentence_ranks
    return sentence_ranks


In [ ]:

pos_summary = summarize(pos_string,3)

In [ ]:
pos_summary

In [ ]:
neg_summary = summarize(neg_string,3)

In [ ]:
neg_summary

In [ ]:
#open text file
text_file = open("/Users/bilge/Desktop/pos_summarized.txt", "w")

#write string to file
text_file.write(pos_summary)

#close file
text_file.close()

In [ ]:
#open text file
text_file = open("/Users/bilge/Desktop/neg_summarized.txt", "w")

#write string to file
text_file.write(neg_summary)

#close file
text_file.close()